In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [3]:
df = pd.read_csv('../../Data/raw/bangkok_traffy.csv')
df

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787021,KDDNBP,{ร้องเรียน},เขตหนองจอก,ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.86304,13.85633",เขตหนองจอก กรุงเทพมหานคร : VV47+G6M แขวงกระทุ่...,กระทุ่มราย,หนองจอก,กรุงเทพมหานคร,2025-01-16 02:50:24.112172+00,กำลังดำเนินการ,NaN,0,2025-01-16 02:50:26.781334+00
787022,RYM99G,"{ความสะอาด,ถนน}","เขตคลองสาน,ฝ่ายสิ่งแวดล้อมฯ เขตคลองสาน",ปัญหา: บริเวณตลาดพระเครื่องพญาไม้ พบร้านอาหารต...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.49651,13.73469",39 ถ. สมเด็จเจ้าพระยา แขวงสมเด็จเจ้าพระยา เขตค...,สมเด็จเจ้าพระยา,คลองสาน,กรุงเทพมหานคร,2025-01-16 02:51:28.8957+00,กำลังดำเนินการ,NaN,0,2025-01-16 02:53:23.119455+00
787023,2025-BDQ87Y,"{ร้องเรียน,ต้นไม้,ความปลอดภัย}",เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือน...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53975,13.76426",10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพม...,ถนนพญาไท,ราชเทวี,กรุงเทพมหานคร,2025-01-16 02:52:33.878797+00,รอรับเรื่อง,NaN,0,2025-01-16 02:52:38.17364+00
787024,EZCZZU,{ถนน},เขตจอมทอง,ปัญหา: พบโรงกลึง (ไม่ทราบชื่อ) ทำงานบนพื้นผิวจ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.46226,13.67672",123 แขวงบางมด เขตจอมทอง กรุงเทพมหานคร 10150 ปร...,บางมด,จอมทอง,กรุงเทพมหานคร,2025-01-16 02:53:27.583947+00,กำลังดำเนินการ,NaN,0,2025-01-16 02:53:30.250517+00


In [4]:
df.dropna(subset = ['type','district','coords'], inplace = True) # delete sample not have type
df = df[df['type'] != "{}"]
df['type'].value_counts()

type
{ถนน}                                      85538
{ทางเท้า}                                  54467
{แสงสว่าง}                                 34152
{ความสะอาด}                                32213
{กีดขวาง}                                  28392
                                           ...  
{ท่อระบายน้ำ,ถนน,เสนอแนะ,ทางเท้า}              1
{เสียงรบกวน,สะพาน,ความปลอดภัย,ทางเท้า}         1
{กีดขวาง,ท่อระบายน้ำ,น้ำท่วม,ร้องเรียน}        1
{สะพาน,ถนน,เสียงรบกวน,คลอง,ป้าย}               1
{กีดขวาง,ถนน,ทางเท้า,เสียงรบกวน}               1
Name: count, Length: 17459, dtype: int64

In [5]:
bkk_alias = {
    "กรุงเทพมหานคร",
    "จังหวัดกรุงเทพมหานคร",
    "จังหวัดจังหวัด กรุงเทพมหานคร",
    "จังหวัดBangkok",
    "จังหวัดกรุงเทพฯ",
    "Bangkok",
    "กทม",
}
# scope only bangkok
mask = df['province'].isin(bkk_alias)
df = df[mask].assign(province="กรุงเทพมหานคร").reset_index(drop=True)
df.province.value_counts()

province
กรุงเทพมหานคร    670569
Name: count, dtype: int64

In [6]:
df['district'].value_counts()

district
จตุจักร              35617
ประเวศ               23386
วัฒนา                21570
บางกะปิ              21396
คลองเตย              21316
บางแค                19574
ปทุมวัน              18783
บางเขน               18761
ลาดกระบัง            17580
บางขุนเทียน          16943
สวนหลวง              16936
ราชเทวี              16730
ดินแดง               16353
บางกอกน้อย           15916
พระนคร               15551
ห้วยขวาง             15287
บางรัก               14985
บึงกุ่ม              14871
บางซื่อ              14546
สาทร                 13838
คลองสามวา            13774
ธนบุรี               13501
ลาดพร้าว             13150
บางนา                12898
หนองจอก              12783
พญาไท                12709
วังทองหลาง           12648
บางพลัด              11708
มีนบุรี              11678
สายไหม               11669
ภาษีเจริญ            11542
ตลิ่งชัน             11161
ยานนาวา              11030
ป้อมปราบศัตรูพ่าย    10632
พระโขนง              10482
หลักสี่              10196
จอมทอง             

In [7]:
top8 = df['district'].value_counts().head(8).index
top8

Index(['จตุจักร', 'ประเวศ', 'วัฒนา', 'บางกะปิ', 'คลองเตย', 'บางแค', 'ปทุมวัน',
       'บางเขน'],
      dtype='object', name='district')

In [8]:
df['coords'].head(10)


0    100.66709,13.67891
1    100.52649,13.72060
2    100.59165,13.82280
3    100.64690,13.67083
4    100.65440,13.68158
5    100.68837,13.71887
6    100.53764,13.70716
7    100.53025,13.70566
8    100.52970,13.70569
9    100.52760,13.70615
Name: coords, dtype: object

In [9]:
df[['lon', 'lat']] = df['coords'].str.split(',', expand=True).astype(float)

results = []

for d in top8:
    sub = df[df['district'] == d].copy()

    # ใช้ lat, lon ในการ cluster
    X = sub[['lat', 'lon']].values

    # --- Run DBSCAN ---
    db = DBSCAN(eps=0.005, min_samples=5).fit(X)
    sub['cluster'] = db.labels_

    # --- กรอง noise ---
    sub = sub[sub['cluster'] != -1]

    # เก็บผล
    results.append(sub)

df_clean = pd.concat(results, ignore_index=True)

df_clean.head()


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,lon,lat,cluster
0,2022-8QL8RE,{ถนน},"เขตจตุจักร,สน.พหลโยธิน,สำนักงานตำรวจแห่งชาติ (...",ถนน,https://storage.googleapis.com/traffy_public_b...,NaN,"100.56270,13.81236",93 ถ. ลาดพร้าว แขวง จอมพล เขตจตุจักร กรุงเทพมห...,จอมพล,จตุจักร,กรุงเทพมหานคร,2022-04-10 16:14:18.780788+00,กำลังดำเนินการ,NaN,0,2024-03-05 23:39:54.014906+00,100.56270,13.81236,0
1,2022-9GW7CE,{ถนน},"เขตจตุจักร,สำนักการโยธา กทม.,กองควบคุมการก่อสร...",ฝนตกถนนลื่นขี้เลนตรวจสอบ หน่วยงานไหนเป็นผู้รับ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.58576,13.85828",2673 ถ. พหลโยธิน แขวง ลาดยาว เขตจตุจักร กรุงเท...,ลาดยาว,จตุจักร,กรุงเทพมหานคร,2022-04-30 04:57:49.906226+00,เสร็จสิ้น,3.0,0,2024-05-20 13:05:34.528824+00,100.58576,13.85828,0
2,2022-UK897G,"{ทางเท้า,ถนน}",โครงการรถไฟฟ้าสายสีเขียวเหนือ (ถนนพหลโยธิน และ...,ทางเท้าสภาพแตกร้าว ถ้าเป็นผู้พอการไม่น่าจะผ่าน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56301,13.81809",1194/9 ถ. พหลโยธิน แขวง จอมพล เขตจตุจักร กรุงเ...,จตุจักร,จตุจักร,กรุงเทพมหานคร,2022-05-02 10:24:20.109196+00,กำลังดำเนินการ,NaN,1,2024-11-04 04:09:50.716992+00,100.56301,13.81809,0
3,2022-K8W922,{ความปลอดภัย},"เขตจตุจักร,ฝ่ายโยธา เขตจตุจักร",พบเห็นการก่อสร้างบริเวณหน้าโครงการ The Crest P...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56053,13.81382",2 ถ. พหลโยธิน แขวง จอมพล เขตจตุจักร กรุงเทพมหา...,จอมพล,จตุจักร,กรุงเทพมหานคร,2022-05-02 11:29:14.387136+00,เสร็จสิ้น,NaN,0,2024-03-21 05:15:10.914103+00,100.56053,13.81382,0
4,2022-746HP4,{ทางเท้า},"เขตจตุจักร,ฝ่ายเทศกิจ เขตจตุจักร",พบเห็นป้ายหาเสียงตั้งขวางทางเท้า เหตุเกิดบนถนน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56150,13.81605",ห้าแยกลาดพร้าว แขวง จตุจักร เขตจตุจักร กรุงเทพ...,จอมพล,จตุจักร,กรุงเทพมหานคร,2022-05-02 11:34:48.855564+00,เสร็จสิ้น,1.0,0,2024-04-17 09:29:37.881862+00,100.56150,13.81605,0


In [10]:
df_clean

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,lon,lat,cluster
0,2022-8QL8RE,{ถนน},"เขตจตุจักร,สน.พหลโยธิน,สำนักงานตำรวจแห่งชาติ (...",ถนน,https://storage.googleapis.com/traffy_public_b...,NaN,"100.56270,13.81236",93 ถ. ลาดพร้าว แขวง จอมพล เขตจตุจักร กรุงเทพมห...,จอมพล,จตุจักร,กรุงเทพมหานคร,2022-04-10 16:14:18.780788+00,กำลังดำเนินการ,NaN,0,2024-03-05 23:39:54.014906+00,100.56270,13.81236,0
1,2022-9GW7CE,{ถนน},"เขตจตุจักร,สำนักการโยธา กทม.,กองควบคุมการก่อสร...",ฝนตกถนนลื่นขี้เลนตรวจสอบ หน่วยงานไหนเป็นผู้รับ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.58576,13.85828",2673 ถ. พหลโยธิน แขวง ลาดยาว เขตจตุจักร กรุงเท...,ลาดยาว,จตุจักร,กรุงเทพมหานคร,2022-04-30 04:57:49.906226+00,เสร็จสิ้น,3.0,0,2024-05-20 13:05:34.528824+00,100.58576,13.85828,0
2,2022-UK897G,"{ทางเท้า,ถนน}",โครงการรถไฟฟ้าสายสีเขียวเหนือ (ถนนพหลโยธิน และ...,ทางเท้าสภาพแตกร้าว ถ้าเป็นผู้พอการไม่น่าจะผ่าน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56301,13.81809",1194/9 ถ. พหลโยธิน แขวง จอมพล เขตจตุจักร กรุงเ...,จตุจักร,จตุจักร,กรุงเทพมหานคร,2022-05-02 10:24:20.109196+00,กำลังดำเนินการ,NaN,1,2024-11-04 04:09:50.716992+00,100.56301,13.81809,0
3,2022-K8W922,{ความปลอดภัย},"เขตจตุจักร,ฝ่ายโยธา เขตจตุจักร",พบเห็นการก่อสร้างบริเวณหน้าโครงการ The Crest P...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56053,13.81382",2 ถ. พหลโยธิน แขวง จอมพล เขตจตุจักร กรุงเทพมหา...,จอมพล,จตุจักร,กรุงเทพมหานคร,2022-05-02 11:29:14.387136+00,เสร็จสิ้น,NaN,0,2024-03-21 05:15:10.914103+00,100.56053,13.81382,0
4,2022-746HP4,{ทางเท้า},"เขตจตุจักร,ฝ่ายเทศกิจ เขตจตุจักร",พบเห็นป้ายหาเสียงตั้งขวางทางเท้า เหตุเกิดบนถนน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56150,13.81605",ห้าแยกลาดพร้าว แขวง จตุจักร เขตจตุจักร กรุงเทพ...,จอมพล,จตุจักร,กรุงเทพมหานคร,2022-05-02 11:34:48.855564+00,เสร็จสิ้น,1.0,0,2024-04-17 09:29:37.881862+00,100.56150,13.81605,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180398,2025-NWRTH4,"{ป้าย,ถนน,ความสะอาด}","เขตบางเขน,ฝ่ายเทศกิจ เขตบางเขน",ซอยเปลื้องอุทิศ ถนนสุขาภิบาล5 ในซอยมีการลักลอบ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.64934,13.87701",337 ถนนสุขาภิบาล ๕ แขวง ท่าแร้ง เขตบางเขน กรุง...,ท่าแร้ง,บางเขน,กรุงเทพมหานคร,2025-01-15 15:21:33.040154+00,กำลังดำเนินการ,NaN,0,2025-01-16 01:36:27.414962+00,100.64934,13.87701,0
180399,2025-G87GPB,{เสียงรบกวน},"เขตบางเขน,ฝ่ายสิ่งแวดล้อมฯ เขตบางเขน",ปัญหาร้านอาหารเปิดดนตรีคาราโอเกะในช่วงเวลาหลัง...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.61927,13.87008",146/2 ซอย รามอินทรา 19 แขวงอนุสาวรีย์ เขตบางเข...,อนุสาวรีย์,บางเขน,กรุงเทพมหานคร,2025-01-15 16:57:07.642895+00,กำลังดำเนินการ,NaN,0,2025-01-16 01:36:44.983451+00,100.61927,13.87008,0
180400,2025-DXYYBM,{แสงสว่าง},"บริษัท สเตฟานี่ จำกัด,เขตบางเขน,สำนักการจราจรแ...",ป้ายรถเมล์ตรงหน้า brow codo มืดมากไม่มีไฟ รบกว...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.60320,13.88692",210 ถ. พหลโยธิน แขวงอนุสาวรีย์ เขตบางเขน กรุงเ...,อนุสาวรีย์,บางเขน,กรุงเทพมหานคร,2025-01-15 22:27:19.875889+00,กำลังดำเนินการ,NaN,0,2025-01-16 01:59:00.892997+00,100.60320,13.88692,0
180401,2025-M4MY8T,{กีดขวาง},"เขตบางเขน,สน.บางเขน,สำนักงานตำรวจแห่งชาติ (Roy...",แท็กซี่จอดกีดขวางป้ายรถเมล์,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.60907,13.89769",VJX5+4J2 แขวงอนุสาวรีย์ เขตบางเขน กรุงเทพมหานค...,อนุสาวรีย์,บางเขน,กรุงเทพมหานคร,2025-01-15 23:59:13.552842+00,เสร็จสิ้น,NaN,0,2025-01-16 01:39:36.152086+00,100.60907,13.89769,0


In [11]:
out_path = "../../Data/clean/bankok_traffy_clean.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")